In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/url-flims/link_flims.csv


# B1 : khai báo thư viện

In [2]:
import numpy as np 
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
from datetime import datetime
import sys
import json
sys.setrecursionlimit(10**7)

# B2:lấy id phim bằng cosole (chạy croll rồi mới download id)

In [3]:
'''
function scrollPage() {
  window.scrollTo(0, document.body.scrollHeight);
}

function loadAllResults() {
  const loadMoreButton = document.querySelector('#__next > main > div.ipc-page-content-container.ipc-page-content-container--center.sc-872d7ac7-0.fqEQWL > div.ipc-page-content-container.ipc-page-content-container--center > section > section > div > section > section > div:nth-child(2) > div > section > div.ipc-page-grid.ipc-page-grid--bias-left.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.sc-619d2eab-0.fOxpqs > div > span > button');
  if (loadMoreButton) {
    loadMoreButton.click();
    setTimeout(loadAllResults, 1); // Đợi trước khi tải thêm
  }
}
loadAllResults();
'''

"\nfunction scrollPage() {\n  window.scrollTo(0, document.body.scrollHeight);\n}\n\nfunction loadAllResults() {\n  const loadMoreButton = document.querySelector('#__next > main > div.ipc-page-content-container.ipc-page-content-container--center.sc-872d7ac7-0.fqEQWL > div.ipc-page-content-container.ipc-page-content-container--center > section > section > div > section > section > div:nth-child(2) > div > section > div.ipc-page-grid.ipc-page-grid--bias-left.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.sc-619d2eab-0.fOxpqs > div > span > button');\n  if (loadMoreButton) {\n    loadMoreButton.click();\n    setTimeout(loadAllResults, 1); // Đợi trước khi tải thêm\n  }\n}\nloadAllResults();\n"

In [4]:
'''

// Lấy tất cả các phần tử có class là 'ipc-lockup-overlay' và 'ipc-focusable'
const elements = document.querySelectorAll('.ipc-lockup-overlay.ipc-focusable');

// Tạo một mảng để lưu dữ liệu
const data = [];

// Lặp qua từng phần tử và lấy giá trị của thuộc tính 'href'
elements.forEach(element => {
  const hrefValue = element.getAttribute('href');
  data.push(hrefValue);
});

// Tạo chuỗi CSV từ mảng dữ liệu
const csvContent = 'data:text/csv;charset=utf-8,' + data.join('\n');

// Tạo một liên kết Data URI để tải file CSV
const encodedUri = encodeURI(csvContent);

// Tạo một phần tử a để tạo liên kết và kích hoạt tải về
const link = document.createElement('a');
link.setAttribute('href', encodedUri);
link.setAttribute('download', 'output.csv');
document.body.appendChild(link);
link.click();

'''
    

"\n\n// Lấy tất cả các phần tử có class là 'ipc-lockup-overlay' và 'ipc-focusable'\nconst elements = document.querySelectorAll('.ipc-lockup-overlay.ipc-focusable');\n\n// Tạo một mảng để lưu dữ liệu\nconst data = [];\n\n// Lặp qua từng phần tử và lấy giá trị của thuộc tính 'href'\nelements.forEach(element => {\n  const hrefValue = element.getAttribute('href');\n  data.push(hrefValue);\n});\n\n// Tạo chuỗi CSV từ mảng dữ liệu\nconst csvContent = 'data:text/csv;charset=utf-8,' + data.join('\n');\n\n// Tạo một liên kết Data URI để tải file CSV\nconst encodedUri = encodeURI(csvContent);\n\n// Tạo một phần tử a để tạo liên kết và kích hoạt tải về\nconst link = document.createElement('a');\nlink.setAttribute('href', encodedUri);\nlink.setAttribute('download', 'output.csv');\ndocument.body.appendChild(link);\nlink.click();\n\n"

In [5]:
import re
def convert_duration_to_minutes(duration_str):
    # Sử dụng biểu thức chính quy để tìm các phần tử thời gian trong chuỗi
    match = re.match(r'PT(\d+H)?(\d+M)?', duration_str)

    if match:
        hours = int(match.group(1)[:-1]) if match.group(1) else 0
        minutes = int(match.group(2)[:-1]) if match.group(2) else 0

        total_minutes = hours * 60 + minutes
        return total_minutes

    return None

In [6]:
# hàm cào thông tin phim
def Scraping_data(url):
    HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}
    req = requests.get(url,headers=HEADERS)
    soup = BeautifulSoup(req.text, 'html')
    data = {}
    script_tag = soup.find('script' , type="application/ld+json")
    elements = json.loads(script_tag.string)
    #print(elements)
    # tìm tên năm sản xuất , độ tuổi , thời lượng phim
    data['Name'] = elements['name']
    if 'datePublished' in elements :
        data['Release Year'] = elements['datePublished'][:4]
    else:
        return None
    data['Certificate'] = None if (elements is None) or (elements.get('contentRating') is None) else elements['contentRating']
    # thời gian của phim
    data['Duration'] = None
    if 'duration' in elements :
        data['Duration'] = convert_duration_to_minutes(elements['duration'])

    # link poster 
    data['Poster'] = elements['image']
    if 'trailer' in elements:
        data['Trailer'] = elements['trailer']['url']
    else :
        data['Trailer'] = None
    data['Genre'] = elements['genre']
    data['Rating'] = elements['aggregateRating']['ratingValue']
    data['Rating Count'] = elements['aggregateRating']['ratingCount']
    
    # tìm nước sản xuất 
    origins =  soup.find('li' ,{'class': 'ipc-metadata-list__item', 'data-testid': 'title-details-origin','role' : "presentation"}).find_all('a')
    data['Country'] = [origin.text for origin in origins]
    
    # tìm đạo diễn , biên kịch
    data['Director'] = [director['name'] for director in elements['director'] if '@type' in director and director['@type'] == 'Person']
    if 'creator' in elements:
        data['Writer'] = [creator['name'] for creator in elements['creator'] if '@type' in creator and creator['@type'] == 'Person']
    else :
        data['Writer'] = data['Director']
    if 'actor' in elements :
        data['Stars'] = [actor['name'] for actor in elements['actor']]
    else :
        data['Stars'] = None 
    data['Story'] = None
    if 'description'in elements :
        data['Story'] = elements['description']
    
    # chi phí , doanh thu bộ flim
    Box_office = soup.find_all('li',class_ = "ipc-metadata-list__item sc-ae184607-2 jqHug")
    data['Budget'] = None
    data['Gross US & Canada'] = None
    data['Opening weekend US & Canada'] = None
    data['Gross worldwide'] = None
    
    if Box_office is not None:
        for i in range(0,len(Box_office)) :
            data[Box_office[i].find_all('span')[0].text] = Box_office[i].find_all('span')[1].text
        #data['Income'] = soup.find_all('li',class_ = "ipc-metadata-list__item sc-ae184607-2 jqHug")[3].find_all('span')[1].text[1:]
    return data

In [7]:
# đọc file url_flim

urls = pd.read_csv('/kaggle/input/url-flims/link_flims.csv',names = ['url'])
urls.url = 'https://www.imdb.com/' + urls.url

In [ ]:
# cào data
data_flims = []

for url in urls.url :
    print(url)
    data = Scraping_data(url)
    if data is not None:
        data_flims.append(data)



https://www.imdb.com//title/tt1392170/?ref_=sr_i_1
https://www.imdb.com//title/tt1951264/?ref_=sr_i_2
https://www.imdb.com//title/tt0446029/?ref_=sr_i_3
https://www.imdb.com//title/tt1649419/?ref_=sr_i_4
https://www.imdb.com//title/tt0993846/?ref_=sr_i_5
https://www.imdb.com//title/tt0361748/?ref_=sr_i_6
https://www.imdb.com//title/tt2582802/?ref_=sr_i_7
https://www.imdb.com//title/tt1375666/?ref_=sr_i_8
https://www.imdb.com//title/tt1038686/?ref_=sr_i_9
https://www.imdb.com//title/tt1981677/?ref_=sr_i_10
https://www.imdb.com//title/tt1285016/?ref_=sr_i_11
https://www.imdb.com//title/tt1446714/?ref_=sr_i_12


In [ ]:
df = pd.DataFrame(data_flims)
df

In [ ]:
df.to_csv('data_flim.csv',index = False)